In [ ]:
# Author María Martínez
# Creation Date 051023 

import pandas as pd
import numpy as np
%matplotlib inline

sns.set()
import warnings
warnings.filterwarnings('ignore')

In [76]:
# Read the dataset
df=pd.read_csv('Dataset/AccidentesAviones.csv')
df.head()

,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [ ]:
# Some columns have values '?', so they are null values
df.info()

In [ ]:
# First I'm going to dois replace all values "?" by NaN 
# Then I'm going to delete columns that do not work in this context
# flight_no, registratio, cn_ln (It seems is a serie number) and summary, that is an event description.
df.replace('?',np.nan,inplace=True)
df.drop(columns=['flight_no','registration','cn_ln','summary'],inplace=True)

In [ ]:
# Check again the null values
df.isna().sum()

In [ ]:
# Delete null value of cantidad de fallecidos, 'cause I am interested in doing an analysis of the mortality rate
# and if there are records that do not have that data, they will not be very useful.
df.dropna(subset='cantidad de fallecidos',inplace=True)
df=df.reset_index(drop=True)

In [ ]:
# Standardize column names
df.columns= df.columns.str.lower()

In [ ]:
# I have a date field that doesn't really help in the analysis because it is a string,
# so I am going to change the format and leave it in YYYY-MM-DD format.
# First, I'm going to separate the data by spaces and assign it to new columns in the dataframe.
df[['mes','dia','anio']]=df['fecha'].str.split(' ', expand=True)

In [ ]:
# Now, I have to change the format of the days left with a comma at the end
# For this I'm going to replace the comma with an empty value
df['dia']=df.dia.str.replace(',','')

In [ ]:
# In the month column I am going to replace the month with the representative number for each month
df['mes']=pd.to_datetime(df['mes'], format='%B').dt.month

In [ ]:
# I will assign the combination of the year, month and day columns to the date column and remove these columns,
# leaving only the date column.
df['fecha']=df['anio'].astype(str)+'-'+df['mes'].astype(str)+'-'+df['dia'].astype(str)
df.drop(columns=['mes','dia','anio'],inplace=True)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# For my analysis I would like to review only the flights that had people on board, 
# so I am going to eliminate the records that do not have them because I think there is inconsistent information.
df[df['all_aboard']=='0']

In [ ]:
df['all_aboard'].loc[30]='1'
df['all_aboard'].loc[119]='2'
df['cantidad de fallecidos'].loc[119]='2'
df['all_aboard'].loc[1114]='2'
df['cantidad de fallecidos'].loc[1114]='2'

In [ ]:
df=df[df['all_aboard']!='0']
df=df.reset_index(drop=True)

In [ ]:
df['route']
# The last thing I see are names of cities and at the beginning the type of flight, 
# I am going to eliminate this column because it seems inconsistent to me
df.drop(columns='route',inplace=True)

In [ ]:
df.head()

In [ ]:
# I see that the hora declarada column does not have a uniform format either,
# I am going to normalize it by putting all the values as 4 consecutive numbers.
# Since I know that the data is in string format, I am going to filter the data 
# by those that have more than 4 characters and use replace to replace the characters I don't want 
# with empty ones.
df[df['hora declarada'].str.len()>4]

In [ ]:
df['hora declarada']=df['hora declarada'].str.replace(':','')
df['hora declarada']=df['hora declarada'].str.replace('c','')

In [ ]:
# This data from which I removed the c has a space before the time, I will remove it with the strip function.
df['hora declarada']=df['hora declarada'].str.strip()

In [ ]:
# I check again what data I have left to normalize
df[df['hora declarada'].str.len()>4]

In [ ]:
df['hora declarada']=df['hora declarada'].str.replace('Z','')
df['hora declarada']=df['hora declarada'].str.replace(';','')

In [ ]:
# I check that there is no data left to normalize and count again how many null values there are to take into account when converting to time format 
# so that no more null values appear
df[df['hora declarada'].str.len()>4]

In [ ]:
df['hora declarada'].isna().sum()

In [ ]:
# Now, to compare values, I am going to put the time with the time format in a new auxiliary column
# in case more NaN values appear when I convert
# df['hora'] = pd.to_datetime(df['hora declarada'], format='%H%M').dt.time
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# I also can't calculate the mortality rate if I don't know how many people were on board in total, 
# I'm going to check the records in this column that appear as NaN
df[df.all_aboard.isna()]

In [ ]:
# I really have no way of knowing how many total people were on board so I'm going to delete these records
df.dropna(subset='all_aboard',inplace=True)
df=df.reset_index(drop=True)
df['fecha']=pd.to_datetime(df['fecha'],format='%Y-%m-%d',errors='coerce')

In [ ]:
df.info()

In [ ]:
# I'm going to rename the columns so that they are all in Spanish.  
df.rename(columns={'ac_type':'tipo_aeronave', 'hora declarada':'hora_declarada','all_aboard':'total_abordo',
                   'pasajeros a bordo':'pasajeros_abordo','crew_aboard': 'tripulacion_abordo', 'cantidad de fallecidos':'total_fatalidades',
                   'passenger_fatalities':'pasajeros_fatalidades', 'crew_fatalities':'tripulacion_fatalidades', 'ground':'tierra_fatalidades'
}, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df['pasajeros_abordo'].loc[df['pasajeros_abordo'].isna() & df['tripulacion_abordo'].notna()]=df['pasajeros_abordo'].loc[df['pasajeros_abordo'].isna() & df['tripulacion_abordo'].notna()].fillna(0)
df['tripulacion_abordo'].loc[df['pasajeros_abordo'].notna() & df['tripulacion_abordo'].isna()]=df['tripulacion_abordo'].loc[df['pasajeros_abordo'].notna() & df['tripulacion_abordo'].isna()].fillna(0)
df['pasajeros_fatalidades'].loc[df['pasajeros_fatalidades'].isna() & df['tripulacion_fatalidades'].notna()]=df['pasajeros_fatalidades'].loc[df['pasajeros_fatalidades'].isna() & df['tripulacion_fatalidades'].notna()].fillna(df['total_fatalidades'].astype(float)-df['tripulacion_fatalidades'].astype(float))
df['tripulacion_fatalidades'].loc[df['pasajeros_fatalidades'].notna() & df['tripulacion_fatalidades'].isna()]=df['tripulacion_fatalidades'].loc[df['pasajeros_fatalidades'].notna() & df['tripulacion_fatalidades'].isna()].fillna(df['total_fatalidades'].astype(float)-df['pasajeros_fatalidades'].astype(float))

In [ ]:
df.to_csv('AccidentesAereos_ETL.csv')
